Neighborhoods in Toronto:
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [277]:
import pandas as pd
import requests
from lxml import html
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url = requests.get(url)

 website (class "wikitable sortable") scraping: 

In [278]:
content = url.content
html = html.fromstring(content)
packages = html.xpath('//table[@class=\"wikitable sortable\"]//tr/td//text()') 
for i in range(packages.count('\n')):
    packages.remove('\n')
n=3 # divide for 3 columns :
tbl=[packages[i * n:(i + 1) * n] for i in range((len(packages) + n - 1) // n )]  
df=pd.DataFrame(tbl, columns=['PostalCode','Borough','Neighbourhood'])
df.sort_values(by=['Borough','Neighbourhood'], ascending=True,inplace=True)
print(df)

    PostalCode          Borough         Neighbourhood
182        M4S  Central Toronto          Davisville\n
155        M4P  Central Toronto    Davisville North\n
210        M4V  Central Toronto             Deer Park
156        M5P  Central Toronto     Forest Hill North
211        M4V  Central Toronto      Forest Hill SE\n
..         ...              ...                   ...
133        M6M             York          Mount Dennis
146        M6N             York           Runnymede\n
134        M6M             York           Silverthorn
145        M6N             York  The Junction North\n
149        M9N             York                Weston

[287 rows x 3 columns]


Transformation of data frame:
    drop 'Not assigned' data:

In [279]:
rows=df.shape[0]
for row1 in range(rows):
    if df.iat[row1,1]!='Not assigned':
        if df.iat[row1,2].replace('\n','')=='Not assigned':
            df.iat[row1,2]=df.iat[row1,1]
        else: 
            df.iat[row1,2]=df.iat[row1,2].replace('\n','')
    else:
        df.iloc[[row1]]=[[pd.NaT,pd.NaT,pd.NaT]]


combination of Neighbourhoods from one Borough into one row:

In [280]:
boo=False
while(boo==False):
    for row in range(rows):
        if row+1 < rows:
            if (df.iat[row,1]==df.iat[row+1,1]):
                df.iloc[[row]]=[[df.iat[row,0], df.iat[row,1], df.iat[row,2].replace('\n','')+', '+df.iat[row+1,2].replace('\n','') ]] #, ignore_index=True)    
                df.iloc[[row+1]]=[[pd.NaT,pd.NaT,pd.NaT]]
    df.dropna(inplace=True) # drop all rows with pd.NaT
    bool_series = df.duplicated(['Borough'],False) 
    boo=df[bool_series].empty
    df.sort_values(by=['Borough'], inplace = True)
    rows=df.shape[0]

In [283]:
# final dataframe (11 rows):
df.reset_index(drop=True, inplace=True)
print(df)
print('Total: ',rows,' rows')

   PostalCode           Borough  \
0         M4S   Central Toronto   
1         M5H  Downtown Toronto   
2         M4E      East Toronto   
3         M4J         East York   
4         M9C         Etobicoke   
5         M7R       Mississauga   
6         M6A        North York   
7         M7A      Queen's Park   
8         M1S       Scarborough   
9         M6J      West Toronto   
10        M6C              York   

                                        Neighbourhood  
0   Davisville, Davisville North, Summerhill West,...  
1   Adelaide, Bathurst Quay, Cabbagetown, Central ...  
2   The Beaches, The Beaches West, The Danforth We...  
3   East Toronto, Leaside, Parkview Hill, Thorncli...  
4   Old Burnhamthorpe, Old Mill North, Montgomery ...  
5               Canada Post Gateway Processing Centre  
6   Lawrence Heights, Lawrence Manor, Willowdale S...  
7                                        Queen's Park  
8   Agincourt, Agincourt North, Woburn, Malvern, M...  
9   Trinity, Swanse